In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
import joblib
import requests
from io import StringIO
import os
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

print("🚀 AI Wellness Assistant - Complete Model Training")
print("="*60)

# Create directories
os.makedirs('models', exist_ok=True)
os.makedirs('data', exist_ok=True)
os.makedirs('visualizations', exist_ok=True)

# ================================================================
# 1. HEART DISEASE MODEL TRAINING (UCI Dataset)
# ================================================================

def train_heart_disease_model():
    print("\n🫀 TRAINING HEART DISEASE MODEL")
    print("-" * 40)

    # Load UCI Heart Disease Dataset
    print("📥 Loading UCI Heart Disease Dataset...")
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"

    column_names = [
        'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
        'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'
    ]

    try:
        response = requests.get(url)
        heart_data = pd.read_csv(StringIO(response.text), names=column_names, na_values='?')

        # Clean data
        heart_data = heart_data.dropna()
        heart_data['target'] = (heart_data['target'] > 0).astype(int)

        # Ensure numeric types
        numeric_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca']
        for col in numeric_cols:
            heart_data[col] = pd.to_numeric(heart_data[col], errors='coerce')

        heart_data = heart_data.dropna()

        print(f"✅ Dataset loaded: {heart_data.shape}")
        print(f"✅ Disease cases: {heart_data['target'].sum()} ({heart_data['target'].mean():.1%})")

        # Save dataset
        heart_data.to_csv('data/heart.csv', index=False)
        print("✅ Saved heart.csv for app use")

    except Exception as e:
        print(f"❌ Error loading dataset: {e}")
        return None, None

    # Prepare features and target
    X = heart_data.drop('target', axis=1)
    y = heart_data['target']

    # Feature engineering for XGBoost format
    print("🔄 Engineering features for XGBoost...")

    # One-hot encode categorical features
    X_encoded = pd.get_dummies(X, columns=['cp', 'restecg', 'slope', 'thal'], drop_first=False)

    # Ensure all features are numeric
    X_encoded = X_encoded.astype(float)

    print(f"✅ Feature engineering complete: {X_encoded.shape} features")
    print(f"Features: {list(X_encoded.columns)}")

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_encoded, y, test_size=0.2, random_state=42, stratify=y
    )

    # Train XGBoost model
    print("🤖 Training XGBoost model...")
    xgb_model = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric='logloss'
    )

    xgb_model.fit(X_train, y_train)

    # Evaluate model
    y_pred = xgb_model.predict(X_test)
    y_prob = xgb_model.predict_proba(X_test)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    print(f"📊 XGBoost Heart Disease Results:")
    print(f"   Accuracy:  {accuracy:.3f}")
    print(f"   Precision: {precision:.3f}")
    print(f"   Recall:    {recall:.3f}")
    print(f"   F1-Score:  {f1:.3f}")
    print(f"   AUC:       {auc:.3f}")

    # Save XGBoost model
    xgb_model.save_model('models/xgb_heart.json')
    print("✅ Saved models/xgb_heart.json")

    # Train Logistic Regression for heart age calculation
    print("🤖 Training Logistic Regression for heart age...")

    # Use only age for heart age calculation
    X_age = heart_data[['age']]

    logreg = LogisticRegression(random_state=42, max_iter=1000)
    logreg.fit(X_age, y)

    # Save logistic regression model
    joblib.dump(logreg, 'models/logreg_baseline.joblib')
    print("✅ Saved models/logreg_baseline.joblib")

    return xgb_model, logreg

# ================================================================
# 2. DIABETES MODEL TRAINING (Pima Indians Dataset)
# ================================================================

def train_diabetes_model():
    print("\n🩸 TRAINING DIABETES MODEL")
    print("-" * 40)

    # Load Pima Indians Diabetes Dataset
    print("📥 Loading Pima Indians Diabetes Dataset...")
    url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

    column_names = [
        'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
        'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'
    ]

    try:
        diabetes_data = pd.read_csv(url, names=column_names)

        # Clean data - replace 0 with NaN for certain columns
        zero_not_accepted = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

        for column in zero_not_accepted:
            diabetes_data[column] = diabetes_data[column].replace(0, np.nan)
            diabetes_data[column].fillna(diabetes_data[column].median(), inplace=True)

        print(f"✅ Dataset loaded: {diabetes_data.shape}")
        print(f"✅ Diabetes cases: {diabetes_data['Outcome'].sum()} ({diabetes_data['Outcome'].mean():.1%})")

    except Exception as e:
        print(f"❌ Error loading dataset: {e}")
        return None

    # Prepare features and target
    X = diabetes_data.drop('Outcome', axis=1)
    y = diabetes_data['Outcome']

    print(f"✅ Features: {list(X.columns)}")

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Train XGBoost model
    print("🤖 Training XGBoost model...")
    xgb_model = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric='logloss'
    )

    xgb_model.fit(X_train, y_train)

    # Evaluate model
    y_pred = xgb_model.predict(X_test)
    y_prob = xgb_model.predict_proba(X_test)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    print(f"📊 XGBoost Diabetes Results:")
    print(f"   Accuracy:  {accuracy:.3f}")
    print(f"   Precision: {precision:.3f}")
    print(f"   Recall:    {recall:.3f}")
    print(f"   F1-Score:  {f1:.3f}")
    print(f"   AUC:       {auc:.3f}")

    # Save XGBoost model
    xgb_model.save_model('models/xgb_diabetes.json')
    print("✅ Saved models/xgb_diabetes.json")

    return xgb_model

# ================================================================
# 3. HYPERTENSION MODEL TRAINING (Kaggle Dataset)
# ================================================================

def train_hypertension_model():
    print("\n🩺 TRAINING HYPERTENSION MODEL")
    print("-" * 40)

    # Upload Kaggle hypertension dataset
    print("📤 Please upload the hypertension dataset (CSV file):")
    print("Download from: https://www.kaggle.com/datasets/ankushpanday1/hypertension-risk-prediction-dataset")

    uploaded = files.upload()

    # Find the uploaded CSV file
    csv_file = None
    for filename in uploaded.keys():
        if filename.endswith('.csv'):
            csv_file = filename
            break

    if csv_file is None:
        print("❌ No CSV file uploaded. Creating synthetic hypertension dataset...")
        return create_synthetic_hypertension_model()

    try:
        hypertension_data = pd.read_csv(csv_file)
        print(f"✅ Dataset loaded: {hypertension_data.shape}")
        print(f"✅ Columns: {list(hypertension_data.columns)}")

        # Find target column
        target_candidates = ['hypertension', 'Hypertension', 'target', 'label', 'risk', 'Risk']
        target_col = None

        for col in target_candidates:
            if col in hypertension_data.columns:
                target_col = col
                break

        if target_col is None:
            print("❌ No target column found. Using last column as target.")
            target_col = hypertension_data.columns[-1]

        print(f"✅ Using '{target_col}' as target column")

        # Prepare data
        X = hypertension_data.drop(target_col, axis=1)
        y = hypertension_data[target_col]

        # Handle categorical variables
        categorical_cols = X.select_dtypes(include=['object']).columns

        if len(categorical_cols) > 0:
            print(f"🔄 Encoding categorical columns: {list(categorical_cols)}")
            X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=False)
        else:
            X_encoded = X.copy()

        # Ensure all features are numeric
        X_encoded = X_encoded.astype(float)

        print(f"✅ Final feature count: {X_encoded.shape}")
        print(f"✅ Hypertension cases: {y.sum()} ({y.mean():.1%})")

    except Exception as e:
        print(f"❌ Error processing dataset: {e}")
        return create_synthetic_hypertension_model()

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_encoded, y, test_size=0.2, random_state=42, stratify=y
    )

    # Train XGBoost model
    print("🤖 Training XGBoost model...")
    xgb_model = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric='logloss'
    )

    xgb_model.fit(X_train, y_train)

    # Evaluate model
    y_pred = xgb_model.predict(X_test)
    y_prob = xgb_model.predict_proba(X_test)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    print(f"📊 XGBoost Hypertension Results:")
    print(f"   Accuracy:  {accuracy:.3f}")
    print(f"   Precision: {precision:.3f}")
    print(f"   Recall:    {recall:.3f}")
    print(f"   F1-Score:  {f1:.3f}")
    print(f"   AUC:       {auc:.3f}")

    # Save XGBoost model
    xgb_model.save_model('models/xgb_hypertension.json')
    print("✅ Saved models/xgb_hypertension.json")

    return xgb_model

def create_synthetic_hypertension_model():
    """Create synthetic hypertension dataset and model"""
    print("🎯 Creating synthetic hypertension dataset...")

    np.random.seed(42)
    n_samples = 2000

    # Create comprehensive feature set matching expected format
    feature_names = [
        'Age', 'BMI', 'Cholesterol', 'Systolic_BP', 'Diastolic_BP',
        'Alcohol_Intake', 'Stress_Level', 'Salt_Intake', 'Sleep_Duration', 'Heart_Rate',
        'LDL', 'HDL', 'Triglycerides', 'Glucose', 'Country_Australia', 'Country_Brazil',
        'Country_Canada', 'Country_China', 'Country_France', 'Country_Germany', 'Country_India',
        'Country_Indonesia', 'Country_Italy', 'Country_Japan', 'Country_Mexico', 'Country_Russia',
        'Country_Saudi Arabia', 'Country_South Africa', 'Country_South Korea', 'Country_Spain',
        'Country_Turkey', 'Country_UK', 'Country_USA', 'Smoking_Status_Former', 'Smoking_Status_Never',
        'Physical_Activity_Level_Low', 'Physical_Activity_Level_Moderate', 'Family_History_Yes',
        'Diabetes_Yes', 'Gender_Male', 'Education_Level_Secondary', 'Education_Level_Tertiary',
        'Employment_Status_Retired', 'Employment_Status_Unemployed'
    ]

    # Generate realistic data
    data = {}

    # Continuous variables
    data['Age'] = np.random.normal(50, 15, n_samples).clip(20, 80)
    data['BMI'] = np.random.normal(26, 4, n_samples).clip(18, 40)
    data['Cholesterol'] = np.random.normal(200, 40, n_samples).clip(120, 350)
    data['Systolic_BP'] = np.random.normal(130, 20, n_samples).clip(90, 200)
    data['Diastolic_BP'] = np.random.normal(85, 15, n_samples).clip(60, 140)
    data['Alcohol_Intake'] = np.random.randint(0, 4, n_samples)
    data['Stress_Level'] = np.random.randint(1, 11, n_samples)
    data['Salt_Intake'] = np.random.randint(0, 4, n_samples)
    data['Sleep_Duration'] = np.random.normal(7, 1.5, n_samples).clip(4, 10)
    data['Heart_Rate'] = np.random.normal(75, 12, n_samples).clip(50, 120)
    data['LDL'] = np.random.normal(120, 30, n_samples).clip(50, 250)
    data['HDL'] = np.random.normal(50, 15, n_samples).clip(20, 100)
    data['Triglycerides'] = np.random.normal(150, 50, n_samples).clip(50, 400)
    data['Glucose'] = np.random.normal(100, 25, n_samples).clip(70, 200)

    # Binary/categorical variables
    countries = ['Australia', 'Brazil', 'Canada', 'China', 'France', 'Germany', 'India',
                'Indonesia', 'Italy', 'Japan', 'Mexico', 'Russia', 'Saudi Arabia',
                'South Africa', 'South Korea', 'Spain', 'Turkey', 'UK', 'USA']

    # One-hot encode country (only one country per person)
    for country in countries:
        data[f'Country_{country}'] = 0

    # Randomly assign countries
    selected_countries = np.random.choice(countries, n_samples)
    for i, country in enumerate(selected_countries):
        data[f'Country_{country}'] = 1

    # Other binary features
    data['Smoking_Status_Former'] = np.random.binomial(1, 0.2, n_samples)
    data['Smoking_Status_Never'] = np.random.binomial(1, 0.6, n_samples)
    data['Physical_Activity_Level_Low'] = np.random.binomial(1, 0.3, n_samples)
    data['Physical_Activity_Level_Moderate'] = np.random.binomial(1, 0.4, n_samples)
    data['Family_History_Yes'] = np.random.binomial(1, 0.3, n_samples)
    data['Diabetes_Yes'] = np.random.binomial(1, 0.1, n_samples)
    data['Gender_Male'] = np.random.binomial(1, 0.5, n_samples)
    data['Education_Level_Secondary'] = np.random.binomial(1, 0.4, n_samples)
    data['Education_Level_Tertiary'] = np.random.binomial(1, 0.3, n_samples)
    data['Employment_Status_Retired'] = np.random.binomial(1, 0.2, n_samples)
    data['Employment_Status_Unemployed'] = np.random.binomial(1, 0.1, n_samples)

    # Create DataFrame
    df = pd.DataFrame(data)

    # Generate realistic hypertension target
    risk_score = (
        (df['Age'] - 40) * 0.02 +
        (df['BMI'] - 25) * 0.05 +
        (df['Systolic_BP'] - 120) * 0.03 +
        (df['Diastolic_BP'] - 80) * 0.04 +
        df['Smoking_Status_Former'] * 0.3 +
        (1 - df['Smoking_Status_Never']) * 0.5 +
        df['Physical_Activity_Level_Low'] * 0.4 +
        df['Family_History_Yes'] * 0.6 +
        df['Diabetes_Yes'] * 0.8 +
        (df['Stress_Level'] - 5) * 0.1
    )

    # Convert to probabilities and generate binary target
    probabilities = 1 / (1 + np.exp(-risk_score + 2))
    y = np.random.binomial(1, probabilities, n_samples)

    print(f"✅ Synthetic dataset created: {df.shape}")
    print(f"✅ Hypertension prevalence: {y.mean():.1%}")

    # Train model
    X_train, X_test, y_train, y_test = train_test_split(
        df, y, test_size=0.2, random_state=42, stratify=y
    )

    xgb_model = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        eval_metric='logloss'
    )

    xgb_model.fit(X_train, y_train)

    # Evaluate
    y_pred = xgb_model.predict(X_test)
    y_prob = xgb_model.predict_proba(X_test)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    print(f"📊 Synthetic Hypertension Model Results:")
    print(f"   Accuracy: {accuracy:.3f}")
    print(f"   AUC:      {auc:.3f}")

    # Save model
    xgb_model.save_model('models/xgb_hypertension.json')
    print("✅ Saved models/xgb_hypertension.json")

    return xgb_model

# ================================================================
# 4. CREATE VISUALIZATION AND SUMMARY
# ================================================================

def create_model_summary():
    print("\n📊 CREATING MODEL SUMMARY")
    print("-" * 40)

    # Test model loading
    models_info = {}

    # Test XGBoost models
    for disease in ['heart', 'diabetes', 'hypertension']:
        try:
            model_path = f'models/xgb_{disease}.json'
            if os.path.exists(model_path):
                model = xgb.XGBClassifier()
                model.load_model(model_path)
                models_info[disease] = {'status': 'Loaded', 'type': 'XGBoost'}
                print(f"✅ {disease.title()} XGBoost model: Ready")
            else:
                models_info[disease] = {'status': 'Missing', 'type': 'XGBoost'}
                print(f"❌ {disease.title()} XGBoost model: Missing")
        except Exception as e:
            models_info[disease] = {'status': f'Error: {e}', 'type': 'XGBoost'}
            print(f"❌ {disease.title()} XGBoost model: Error")

    # Test Logistic Regression
    try:
        if os.path.exists('models/logreg_baseline.joblib'):
            logreg = joblib.load('models/logreg_baseline.joblib')
            models_info['logistic_regression'] = {'status': 'Loaded', 'type': 'Logistic Regression'}
            print("✅ Logistic Regression (heart age): Ready")
        else:
            models_info['logistic_regression'] = {'status': 'Missing', 'type': 'Logistic Regression'}
            print("❌ Logistic Regression (heart age): Missing")
    except Exception as e:
        models_info['logistic_regression'] = {'status': f'Error: {e}', 'type': 'Logistic Regression'}
        print("❌ Logistic Regression (heart age): Error")

    # Check data files
    if os.path.exists('data/heart.csv'):
        print("✅ Heart disease dataset (heart.csv): Available")
    else:
        print("❌ Heart disease dataset (heart.csv): Missing")

    return models_info

def create_download_package():
    print("\n📦 CREATING DOWNLOAD PACKAGE")
    print("-" * 40)

    import zipfile

    zip_path = 'trained_models_complete.zip'

    with zipfile.ZipFile(zip_path, 'w') as zipf:
        # Add model files
        model_files = [
            'models/xgb_heart.json',
            'models/xgb_diabetes.json',
            'models/xgb_hypertension.json',
            'models/logreg_baseline.joblib'
        ]

        for file_path in model_files:
            if os.path.exists(file_path):
                zipf.write(file_path, file_path)
                print(f"✅ Added {file_path}")
            else:
                print(f"⚠️ Missing {file_path}")

        # Add data files
        if os.path.exists('data/heart.csv'):
            zipf.write('data/heart.csv', 'data/heart.csv')
            print("✅ Added data/heart.csv")

    print(f"📁 Package created: {zip_path}")
    print(f"📏 Size: {os.path.getsize(zip_path) / (1024*1024):.1f} MB")

    # Download the package
    files.download(zip_path)

    return zip_path

# ================================================================
# 5. MAIN TRAINING EXECUTION
# ================================================================

def main():
    print("🚀 Starting Complete Model Training Pipeline")
    print("=" * 60)

    models_trained = {}

    # Train Heart Disease Model
    try:
        heart_xgb, heart_logreg = train_heart_disease_model()
        models_trained['heart_disease'] = heart_xgb is not None
        models_trained['logistic_regression'] = heart_logreg is not None
    except Exception as e:
        print(f"❌ Heart disease training failed: {e}")
        models_trained['heart_disease'] = False
        models_trained['logistic_regression'] = False

    # Train Diabetes Model
    try:
        diabetes_xgb = train_diabetes_model()
        models_trained['diabetes'] = diabetes_xgb is not None
    except Exception as e:
        print(f"❌ Diabetes training failed: {e}")
        models_trained['diabetes'] = False

    # Train Hypertension Model
    try:
        hypertension_xgb = train_hypertension_model()
        models_trained['hypertension'] = hypertension_xgb is not None
    except Exception as e:
        print(f"❌ Hypertension training failed: {e}")
        models_trained['hypertension'] = False

    # Create summary
    models_info = create_model_summary()

    # Create download package
    zip_path = create_download_package()

    # Final summary
    print("\n" + "=" * 60)
    print("🏆 TRAINING COMPLETE - FINAL SUMMARY")
    print("=" * 60)

    total_models = len([v for v in models_trained.values() if v])
    print(f"📊 Models successfully trained: {total_models}/4")

    for model_name, success in models_trained.items():
        status = "✅ SUCCESS" if success else "❌ FAILED"
        print(f"   {model_name.replace('_', ' ').title()}: {status}")

    print(f"\n📁 All files packaged in: {zip_path}")
    print("📥 Package downloaded automatically")


# Run the complete training pipeline
if __name__ == "__main__":
    main()


🚀 AI Wellness Assistant - Complete Model Training
🚀 Starting Complete Model Training Pipeline

🫀 TRAINING HEART DISEASE MODEL
----------------------------------------
📥 Loading UCI Heart Disease Dataset...
✅ Dataset loaded: (297, 14)
✅ Disease cases: 137 (46.1%)
✅ Saved heart.csv for app use
🔄 Engineering features for XGBoost...
✅ Feature engineering complete: (297, 22) features
Features: ['age', 'sex', 'trestbps', 'chol', 'fbs', 'thalach', 'exang', 'oldpeak', 'ca', 'cp_1.0', 'cp_2.0', 'cp_3.0', 'cp_4.0', 'restecg_0.0', 'restecg_1.0', 'restecg_2.0', 'slope_1.0', 'slope_2.0', 'slope_3.0', 'thal_3.0', 'thal_6.0', 'thal_7.0']
🤖 Training XGBoost model...
📊 XGBoost Heart Disease Results:
   Accuracy:  0.867
   Precision: 0.885
   Recall:    0.821
   F1-Score:  0.852
   AUC:       0.920
✅ Saved models/xgb_heart.json
🤖 Training Logistic Regression for heart age...
✅ Saved models/logreg_baseline.joblib

🩸 TRAINING DIABETES MODEL
----------------------------------------
📥 Loading Pima Indians D

Saving hypertension_dataset.csv to hypertension_dataset (1).csv
✅ Dataset loaded: (174982, 23)
✅ Columns: ['Country', 'Age', 'BMI', 'Cholesterol', 'Systolic_BP', 'Diastolic_BP', 'Smoking_Status', 'Alcohol_Intake', 'Physical_Activity_Level', 'Family_History', 'Diabetes', 'Stress_Level', 'Salt_Intake', 'Sleep_Duration', 'Heart_Rate', 'LDL', 'HDL', 'Triglycerides', 'Glucose', 'Gender', 'Education_Level', 'Employment_Status', 'Hypertension']
✅ Using 'Hypertension' as target column
🔄 Encoding categorical columns: ['Country', 'Smoking_Status', 'Physical_Activity_Level', 'Family_History', 'Diabetes', 'Gender', 'Education_Level', 'Employment_Status']
✅ Final feature count: (174982, 52)
❌ Error processing dataset: Could not convert string '...' to numeric
🎯 Creating synthetic hypertension dataset...
✅ Synthetic dataset created: (2000, 44)
✅ Hypertension prevalence: 39.3%
📊 Synthetic Hypertension Model Results:
   Accuracy: 0.630
   AUC:      0.644
✅ Saved models/xgb_hypertension.json

📊 CREATIN

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🏆 TRAINING COMPLETE - FINAL SUMMARY
📊 Models successfully trained: 4/4
   Heart Disease: ✅ SUCCESS
   Logistic Regression: ✅ SUCCESS
   Diabetes: ✅ SUCCESS
   Hypertension: ✅ SUCCESS

📁 All files packaged in: trained_models_complete.zip
📥 Package downloaded automatically
